In [1]:
import json
import os
from openai import OpenAI
from getpass import getpass
from langchain.document_loaders import JSONLoader

In [2]:
MY_API_KEY = getpass("OpenAi API Key:")

In [3]:
client = OpenAI(api_key=MY_API_KEY)

In [4]:
# 공용 프롬프트

kosme_template = {f'''

1. 문제 인식  
1) 제품 및 서비스 개발동기
(자사가 개발(보유)하고 있는 제품 및 서비스에 대한 개발동기 등을 기재)

2) 제품 및 서비스의 목적(필요성)
(제품(서비스)을 구현하고자 하는 목적, 고객의 니즈를 혁신적으로 해결하기 위한 방안 등을 기재)

2. 실현 가능성
1) 제품 및 서비스의 개발 방안
(제품(서비스) 구현정도ㅡ 제작 소요기간 및 제작방법(자체 외주), 추진일정 등을 기재)

3. 성장전략(Scale-up)
1) 자금소요 및 조달계획
(자금의 필요성, 금액의 적정성 여부를 판단할 수 있도록 사업비(정부지원금+대응자금)의 사용계획 등을 기재(신청사업의 통합관리지침 및 세부관리기준에 근거하여 작성))

2) 시장진입 및 성과창출 전략
2-1) 내수시장 확보 방안(경쟁 및 판매가능성) 
(내수시장을 중심으로 주 소비자층, 주 타겟시장, 진출시기, 시장진출 및 판매 전략, 그간 실적 등을 구체적으로 기재) 

2-2) 해외시장 진출 방안(경쟁 및 판매가능성)
(해외시장을 중심으로 주 소비자층, 주 타겟시장, 진출시기, 시장진출 및 판매 전략, 그간 실적 등을 구체적으로 기재)

4. 팀 구성
1) 대표자 및 직원의 보유역량 및 기술보호 노력

2) 사회적 가치 실천계획
(양질의 일자리 창출을 위한 중소기업 성과공유제, 비공유직의 정규직화, 근로시간 단축 등 사회적 가치 실천계획을 기재)

3) 지역특화 아이디어 기반 사업추진계획
(지역에 특화된 산업 및 문화 등을 신기술 또는 최신 트렌드와 결합한 측면에서 기술)

'''
}

In [5]:
# 공용 프롬프트

kosme_pass = {f'''
1. 사업아이템과 관련된 사회적으로 발생하는 문제점을 제시합니다
2. 사회적 문제에 대한 근거를 제시해 주세요
3. 시장 동향(트렌드)과 성장 가능성을 제시합니다.
4. 사업 시작하게 된 동기와 문제인식, 문제 정의를 제시해 주세요,
5. 시장에서 발견한 고객(이용자)의 니즈를 파악해서 제시해 주세요.
6. 시장에서 발견한 고객(이용자)의 pain-point를 파악해서 제시해 주세요.
7. 제품(서비스)을 구현하고자 하는 목적, 고객의 니즈를 혁신적으로 해결하기 위한 방안을 제시해 주세요.    
8. 가능하다면 시장 확장 가능성에 대해서도 제시해 주세요.
              '''}

In [6]:
# 사용자 입력 받기

아이템_명 = input('아이템 명을 입력하세요: ')
아이템_소개 = input('아이템 소개를 입력하세요: ')
제품_및_서비스_개발동기 = input('제품 및 서비스 개발동기를 입력하세요: ')
제품_및_서비스의_목적_필요성 = input('제품 및 서비스의 목적(필요성)을 입력하세요: ')
제품_및_서비스의_개발_방안 = input('제품 및 서비스의 개발 방안을 입력하세요: ')
자금_조달_및_개발_방안 = input('자금 조달 및 개발 방안을 입력하세요: ')
내수시장_확보_방안_경쟁_및_판매가능성 = input('내수시장 확보 방안(경쟁 및 판매가능성)을 입력하세요: ')
해외시장_진출_방안_경쟁_및_판매가능성 = input('해외시장 진출 방안(경쟁 및 판매가능성)을 입력하세요: ')
대표자_및_직원의_보유역량_및_기술보호_노력 = input('대표자 및 직원의 보유역량 및 기술보호 노력을 입력하세요: ')
사회적_가치_실천계획 = input('사회적 가치 실천계획을 입력하세요: ')
지역특화_아이디어_기반_사업추진계획 = input('지역특화 아이디어 기반 사업추진계획을 입력하세요: ')
타겟_고객 = input('타겟 고객: ')
시장_규모 = input('시장 규모을 입력하세요: ')
경쟁사_분석 = input('경쟁사 분석을 입력하세요: ')

# 입력된 내용을 딕셔너리에 저장
item_info = {
    '아이템 명': 아이템_명,
    '아이템 소개': 아이템_소개,
    '제품 및 서비스 개발동기': 제품_및_서비스_개발동기,
    '제품 및 서비스의 목적_필요성': 제품_및_서비스의_목적_필요성,
    '제품 및 서비스의 개발 방안': 제품_및_서비스의_개발_방안,
    '자금 조달 및 개발 방안': 자금_조달_및_개발_방안,
    '내수시장 확보 방안 경쟁 및 판매가능성': 내수시장_확보_방안_경쟁_및_판매가능성,
    '해외시장 진출 방안 경쟁 및 판매가능성': 해외시장_진출_방안_경쟁_및_판매가능성,
    '대표자 및 직원의 보유역량 및 기술보호 노력': 대표자_및_직원의_보유역량_및_기술보호_노력,
    '사회적 가치 실천계획': 사회적_가치_실천계획,
    '지역특화 아이디어 기반 사업추진계획': 지역특화_아이디어_기반_사업추진계획,
    '타겟 고객': 타겟_고객,
    '시장 규모': 시장_규모,
    '경쟁사 분석': 경쟁사_분석
}

In [7]:
item_info

{'아이템 명': 'OCS 기반 수목의료용 케미컬 및 자재 커머스 O2O 플랫폼',
 '아이템 소개': '처방전달시스템을 통해 나무병원, 조경업체, 케미컬사, 조경자재 판매사, 조경담당자가 수목진료 정보를 공유하여 수목관리 산업의 효율화 및 활성화를 도모하는 O2O 플랫폼임',
 '제품 및 서비스 개발동기': '나무의사의 진료기록을 작성, 보관하거나 처방 약품과 자재를 발주 및 관리하는 시스템이 전무해서',
 '제품 및 서비스의 목적_필요성': '나무병원에서 수목의료를 위해 사용하는 농약 및 비료, 자재 및 소모품 등의 재고 관리나 발주를 수작업으로 관리하고 있어 수목의료용 케미컬 및 자재 관리의 효율성이 떨어져서',
 '제품 및 서비스의 개발 방안': '나무의사용 EMR 시스템은 헬스케어 전문기업 시야인사이트 및 한양대학교 공과대학원과 함께 공동 개발하였으며, 향후 OCS 또한 함께 개발해 나갈 예정',
 '자금 조달 및 개발 방안': '시스템 백앤드 개발 일부를 외주맡길 예정이며 시스템 개발 및 서비스 관리 인력 2인을 채용할 계획',
 '내수시장 확보 방안 경쟁 및 판매가능성': '처방단계에서 농약에 대한 우선순위 제품 노출을 통해 농약사로부터 수익 창출. 농자재사로부터는 저마진 공급 계약을 통해 거래 수수료 발생',
 '해외시장 진출 방안 경쟁 및 판매가능성': '미국 산림청의 I-TREE 서비스 등 그린인프라와 관련한 산업이 확대되고 있으나 이를 시각화한 사례는 찾기 힘들다. 이에 병해충 확산 데이터 시각화 기술을 수출할 계획',
 '대표자 및 직원의 보유역량 및 기술보호 노력': "대표는 한양대학교 도시대학원 랜드스케이프어바니즘 전공이며 인천시설공단과 자이에스엔디에서 10년 간 근무하였음.  기술 보호를 위해서는 특허 3건 출원'",
 '사회적 가치 실천계획': '양질의 일자리로 장기간 함께 할 수 있는 체계를 구축할 예정으로 경영성과급을 도입할 계획임',
 '지역특화 아이디어 기반 사업추진계획': '산림이 많은 강원도에서 많이 발생하는 병해충 예방

In [ ]:
# 1. 문제인식 항목 

prompt_1 = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 항목에 대해 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다.
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스 개발동기와 서비스의 목적&필요성 부분만 작성해 주세요.
시장의 필요성과 법제화가 중요합니다.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

문제 인식 1: {item_info['제품 및 서비스 개발동기']}
문제 인식 2: {item_info['제품 및 서비스의 목적_필요성']} 

**반드시 지켜야 할 출력 규칙**
1. 절대 #, *, - 기호 및 마크다운 스타일 절대 금지
2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함

예시 출력:  
1. 문제인식
   1) 제품 및 서비스 개발동기  
      - 본 제품은 시장의 OOO한 문제를 해결하기 위해 개발되었습니다...  

   2) 제품 및 서비스의 목적 및 필요성  
      - 현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며...  
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt_1}],
                                                            temperature=0.7,
                                                            presence_penalty=0.8,
                                                            frequency_penalty=0.5,
                                                            max_tokens = 4000)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안1 = completion.choices[0].message.content
사업계획서_초안1 = 사업계획서_초안1.replace("### ", "")  # 자동으로 붙은 '### '을 제거  
print("사업계획서 초안 :\n", 사업계획서_초안1)

사업계획서 초안 :
 1. 문제 인식
   1) 제품 및 서비스 개발동기  
      - 본 제품은 수목 관리 산업에서 효율성을 높이고자 하는 시장의 명확한 요구를 해결하기 위해 개발되었습니다. 나무병원과 조경업체가 수목 진료 기록을 작성하고 보관하며, 처방된 약품과 자재를 발주하고 관리하는 시스템이 전무한 상황입니다. 이러한 과정은 주로 수작업으로 이루어지며, 이는 시간 소모가 크고 인간의 실수 가능성이 높은 점에서 비효율적입니다. 특히, 수목 진료에 필요한 데이터는 종종 일관성이 없고 체계적으로 공유되지 않으며, 이로 인해 효과적인 의사결정이 어렵습니다. 따라서, 자동화되고 통합된 솔루션의 필요성이 대두되었으며, 이를 해결하기 위한 OCS 기반 플랫폼이 개발 동기가 되었습니다.

   2) 제품 및 서비스의 목적 및 필요성  
      - 현재 시장에서 유사 제품은 수목 의료를 위한 농약 및 비료, 자재와 소모품 등의 재고 관리나 발주를 여전히 수작업으로 처리하고 있어 큰 한계를 가지고 있습니다. 이러한 방법은 정확성과 신속함을 요구하는 현대적 경영 환경에서는 적합하지 않습니다. 결과적으로 업무의 효율성을 현저히 떨어뜨리고 있으며, 이는 비용 증가와 고객 서비스 품질 저하로 이어집니다(출처: "한국조경신문", 2023). 본 서비스를 통해 제공되는 플랫폼은 EMR 시스템과 OCS 기능을 결합하여 나무병원의 진료 정보와 처방 정보를 체계적으로 관리할 수 있도록 합니다. 또한 이를 통해 나무병원과 관련 업체 간의 원활한 정보 교류가 가능해져 전체 산업 생태계의 활성화를 도모하므로 매우 필요합니다.

특히 이러한 기술적 지원은 사용자에게 더 나은 데이터 기반 의사결정을 할 수 있는 도구를 제공하며, 이는 궁극적으로 고객에게 더 나은 서비스를 제공할 뿐만 아니라 운영상의 많은 부분을 최적화하여 비용 절감 효과도 기대됩니다(출처: "산림청 보고서", 2023). 결론적으로 본 플랫폼은 기존 방식의 한계를 극복하고 새로운 가치를 창출할 혁신적인 솔루션입니다.


In [ ]:
# 2. 실현 가능성 항목 

prompt_2 = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 항목에 대해 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스의 개발 방안 부분만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

실현 가능성 1: {item_info['제품 및 서비스의 개발 방안']}

**반드시 지켜야 할 출력 규칙**
1. 절대 #, *, - 기호 및 마크다운 스타일 절대 금지
2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함

예시 출력: 
2. 실현 가능성
   1) 제품 및 서비스의 개발 방안  
      - 본 제품은 시장의 OOO한 문제를 해결하기 위해 개발되었습니다...  

 
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt_2}],
                                                            temperature=0.7,
                                                            presence_penalty=0.8,
                                                            frequency_penalty=0.5,
                                                            max_tokens = 4000)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안2 = completion.choices[0].message.content
사업계획서_초안2 = 사업계획서_초안2.replace("### ", "")  # 자동으로 붙은 '### '을 제거  
print("사업계획서 초안 :\n", 사업계획서_초안2)

사업계획서 초안 :
 2. 실현 가능성  
   1) 제품 및 서비스의 개발 방안  

   본 제품, 즉 OCS 기반 수목의료용 케미컬 및 자재 커머스 O2O 플랫폼은 수목 관리 산업에서 발생하는 비효율성을 해결하기 위해 개발되었습니다. 특히 나무병원과 조경업체들이 처방 약품 및 자재를 발주하고 관리하는 데 있어, 기존에는 수작업에 의존한 비효율적인 시스템이 문제였습니다. 이를 개선하기 위해 나무의사용 EMR 시스템을 헬스케어 전문기업 '시야인사이트'와 한양대학교 공과대학원과 공동 개발하였으며, 향후 OCS(처방전달 시스템) 또한 이들과 협력하여 추진할 계획입니다.

제품 개발 일정은 단계적으로 설정될 예정이며, 초기 단계에서는 EMR 시스템의 완성도를 높이는 것에 집중했습니다. 현재 진행 중인 EMR 시스템은 데이터 입력과 저장 기능을 강화하고 있으며, 사용자 친화적인 인터페이스를 구현하여 실제 현장에서 쉽게 사용할 수 있도록 설계되었습니다. 제작 소요 기간은 총 12개월로 예상되며, 각 단계별로 철저히 모니터링하여 일정 준수를 목표로 하고 있습니다.

추진 방법으로는 내부 인력과 외부 전문 기업의 협력을 통한 하이브리드 방식이 채택되었습니다. 백엔드 시스템 개발 일부는 외주 계약을 통해 진행함으로써 기술적 전문성과 효율성을 동시에 확보할 계획입니다. 이는 복잡한 서버 운영 및 데이터 처리 과정을 원활하게 수행하기 위한 전략적 선택입니다.

향후 OCS 개발은 EMR 시스템 완료 후 바로 착수할 예정이며, 이는 전체 프로젝트 기간 내에서 병행 가능한 범위 내에서 최대한 빠르게 진행될 것입니다. 주요 기능으로는 처방 기록 공유와 진료 데이터를 기반으로 한 자동 발주 및 재고 관리 기능 등이 포함될 예정입니다.

특히 이러한 모든 과정들은 사용자의 피드백을 반영하여 지속적으로 개선해 나갈 것이며, 이에 따라 사용자 경험 중심의 플랫폼 구축을 목표로 하고 있습니다. 따라서 초기 사용자 그룹을 대상으로 파일럿 테스트를 진행하며 실제 사용 환경에서 발생할 수 있는 

In [54]:
# 3. 성장 전략 항목 

prompt_3 = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 항목에 대해 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 시장진입 및 성과창출 전략에서 내수시장 확보 방안(경쟁 및 판매가능성)과 해외시장 진출 방안(경쟁 및 판매가능성), 자금소요 및 조달계획 부분만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

성장 전략 1: {item_info['내수시장 확보 방안 경쟁 및 판매가능성']}
성장 전략 2: {item_info['해외시장 진출 방안 경쟁 및 판매가능성']}
성장 전략 3: {item_info['자금 조달 및 개발 방안']}

**반드시 지켜야 할 출력 규칙**
1. 절대 #, *, - 기호 및 마크다운 스타일 절대 금지
2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함

예시 출력:
3. 성장 전략  
   1) 내수시장 확보 방안 경쟁 및 판매가능성
      - 본 제품은 시장의 OOO한 문제를 해결하기 위해 개발되었습니다...  

   2) 해외시장 진출 방안 경쟁 및 판매가능성
      - 현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며...
      
   3) 자금 조달 및 개발 방안
      - 현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며...
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt_3}],
                                                            temperature=0.7,
                                                            presence_penalty=0.8,
                                                            frequency_penalty=0.5,
                                                            max_tokens = 4000)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안3 = completion.choices[0].message.content
사업계획서_초안3 = 사업계획서_초안3.replace("### ", "")  # 자동으로 붙은 '### '을 제거  
print("사업계획서 초안 :\n", 사업계획서_초안3)

사업계획서 초안 :
 3. 성장 전략

1) 내수시장 확보 방안 경쟁 및 판매가능성
본 제품은 수목의료 산업에서의 비효율성을 해결하기 위해 개발되었습니다. 현재 국내 조경시장은 약 14조 원 규모로 매우 크지만, 나무병원 시장 자체는 상대적으로 작은 1000억 원 규모로 추산되고 있습니다(출처: 한국조경신문). 이 시장에서도 특히 처방 전달 시스템이 부족하여 나무병원의 진료 기록 작성, 보관, 그리고 약품 및 자재 발주의 효율성이 떨어지고 있습니다. 본 OCS 기반 플랫폼은 이러한 문제를 해결하고자 하며, 농약과 자재에 대한 우선순위 노출을 통해 농약사로부터 수익을 창출하고자 합니다. 또한 농자재사와 저마진 공급 계약을 체결함으로써 거래 수수료를 발생시킬 계획입니다. 이는 고객에게 가격 경쟁력을 제공하여 구매 유도를 높일 것으로 기대됩니다.

특히, 국내 타겟 시장은 나무병원, 조경업체 등으로 이들은 기존의 비효율적인 관리 방식을 개선할 필요가 있는 주 소비층입니다. 플랫폼 도입 초기에는 주요 나무병원들과 파트너십을 맺어 사용자 경험을 극대화하고 장기적 관계를 구축할 것입니다. 이를 통해 초기 사용자의 긍정적 피드백을 기반으로 빠른 확장과 추가 고객 유치를 목표로 하고 있습니다.

2) 해외시장 진출 방안 경쟁 및 판매가능성
해외시장에서는 미국 산림청의 I-TREE 서비스와 같은 그린인프라 관련 산업이 확대되고 있으나 병해충 확산 데이터를 시각화하는 구체적인 사례는 드뭅니다(출처: 미국 산림청 보고서). 이러한 갭을 해소하기 위해 당사의 데이터 시각화 기술을 적극 활용할 계획입니다. 특히 북미 시장에서는 환경 보호와 지속 가능한 관리에 대한 관심이 커지고 있어 본 플랫폼의 기술이 큰 가치가 있을 것입니다.

주 타겟 시장은 미국과 캐나다이며, 현지 조경업체와 정부 기관 등이 주요 고객층이 될 것입니다. 현지화된 솔루션 제공을 위해 현지 파트너와 협력하고 해당 국가 규제에 맞춘 시스템 업그레이드를 진행할 예정입니다. 초기 진출 전략으로는 파일럿 프로그램 운영을 

In [56]:
# 4. 팀 구성 항목 

prompt_4 = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 항목에 대해 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 대표자 및 직원의 보유역량 및 기술보호 노력, 사회적 가치 실천계획 부분, 지역특화 아이디어 기반 사업추진계획만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

팀 구성 1: {item_info['대표자 및 직원의 보유역량 및 기술보호 노력']}
팀 구성 2: {item_info['사회적 가치 실천계획']}
팀 구성 3: {item_info['지역특화 아이디어 기반 사업추진계획']}

**반드시 지켜야 할 출력 규칙**
1. 절대 #, *, - 기호 및 마크다운 스타일 절대 금지
2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함

예시 출력: 
4. 팀 구성
1) 대표자 및 직원의 보유역량 및 기술보호 노력
   - 본 제품은 시장의 OOO한 문제를 해결하기 위해 개발되었습니다...  

2) 사회적 가치 실천계획 
   - 현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며... 
    
3) 지역특화 아이디어 기반 사업추진계획 
   - 현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며...  
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt_4}],
                                                            temperature=0.7,
                                                            presence_penalty=0.8,
                                                            frequency_penalty=0.5,
                                                            max_tokens = 4000)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안4 = completion.choices[0].message.content
사업계획서_초안4 = 사업계획서_초안4.replace("### ", "")  # 자동으로 붙은 '### '을 제거  
print("사업계획서 초안 :\n", 사업계획서_초안4)

사업계획서 초안 :
 4. 팀 구성

1) 대표자 및 직원의 보유역량 및 기술보호 노력
   - 본 사업의 대표자는 한양대학교 도시대학원에서 랜드스케이프어바니즘을 전공한 전문가로서, 인천시설공단과 자이에스엔디에서 10년 간 근무하며 쌓아온 실무 경험과 학문적 지식을 겸비하고 있습니다. 이러한 전문성은 수목관리 산업에 대한 깊은 이해와 혁신적인 접근 방식을 가능하게 합니다. 특히, 환경 조경 분야에서의 풍부한 경험은 나무병원 및 조경업체들의 니즈를 정확히 파악하고 적절한 솔루션을 제시하는 데 큰 도움이 됩니다.
   
   - 기술 보호 측면에서는 현재까지 특허 3건을 출원하여 플랫폼의 핵심 기술과 아이디어를 보호하고 있습니다. 이는 경쟁사가 쉽게 모방할 수 없도록 함으로써 시장 내에서 독창성과 차별성을 유지하는 중요한 전략입니다. 또한, 지속적으로 법률 자문을 받아 지적 재산권 관리에 만전을 기하고 있으며, 새로운 기술 개발 시에도 이를 빠르게 특허화하여 기술 보호의 틀을 강화해 나갈 계획입니다.

2) 사회적 가치 실천계획
   - 본 사업은 양질의 일자리 창출을 위한 정책으로 경영성과급 제도를 도입할 예정입니다. 이는 직원들이 기업 성과와 연계된 추가적인 인센티브를 받을 수 있도록 하여 동기 부여를 극대화할 계획입니다. 장기간 함께 할 수 있는 체계를 구축함으로써 조직 내 안정성을 높이고, 직원들의 전문성을 꾸준히 발전시켜 나가는 것을 목표로 하고 있습니다.
   
   - 더불어, 비정규직 직원을 정규직화하여 고용 안정을 도모하고 공정한 노동 환경을 제공할 계획입니다. 이러한 노력을 통해 회사는 지속 가능한 성장뿐만 아니라 사회적 책임도 충실히 이행합니다. 또한 근로시간 단축 등 워라밸 개선 정책도 적극적으로 추진하여 업무 효율성과 직원 만족도를 동시에 추구합니다.

3) 지역특화 아이디어 기반 사업추진계획
   - 강원도는 산림 면적이 넓고 병해충 발생 빈도가 높은 지역 중 하나로 꼽힙니다. 따라서 본 사업은 이러한 지역적 특수성을 반영하여 병해충 예방

In [ ]:
def 출력_사업계획서_초안(*사업계획서_초안):
    for idx, 초안 in enumerate(사업계획서_초안, start=1):
        print(f"사업계획서 초안 {idx} :\n{초안}\n")      

In [48]:
print("사업계획서 초안 :\n", 사업계획서_초안1)
print(사업계획서_초안2)
print(사업계획서_초안3)
print(사업계획서_초안4)

사업계획서 초안 :
 1. 문제인식
   1) 제품 및 서비스 개발동기  
      - 본 제품은 나무의사의 진료기록을 작성하고 보관하거나, 처방 약품과 자재를 발주 및 관리하는 시스템이 부족하다는 문제를 해결하기 위해 개발되었습니다. 현재 수목관리 산업에서는 이러한 작업들이 대부분 수작업으로 이루어지고 있어 데이터의 정확성과 효율성이 떨어지는 문제가 있습니다. 특히 이러한 비효율성은 농약, 비료, 자재 등의 재고 관리를 어렵게 하고, 그로 인해 불필요한 비용이 발생하거나 공급망에 문제가 생길 수 있습니다. 따라서 우리는 OCS 기반의 플랫폼을 통해 나무병원, 조경업체, 케미컬사 등 관련 업계가 효과적으로 정보를 공유하고 협력할 수 있는 솔루션을 제공하고자 합니다.

   2) 제품 및 서비스의 목적 및 필요성  
      - 현재 시장에서 유사 제품은 존재하지 않거나 매우 제한적인 기능만을 제공하고 있습니다. 이로 인해 많은 나무병원이나 조경업체는 여전히 비효율적인 방법으로 업무를 관리해야 하며, 이는 시간과 비용의 낭비로 이어집니다. 본 서비스는 사용자의 니즈를 혁신적으로 해결하기 위해 통합적이고 자동화된 O2O 플랫폼을 제공합니다. 이를 통해 사용자들은 진료 기록, 처방 전달, 재고 관리 등을 보다 쉽게 처리할 수 있으며, 결과적으로 운영 효율성을 크게 향상시킬 수 있습니다. 또한 본 플랫폼은 최신 기술을 활용하여 병해충 확산 데이터를 시각화함으로써 실시간 모니터링과 예측 기능을 강화하며, 이는 산업 전반에 걸쳐 더욱 신속하고 정확한 의사결정을 가능하게 합니다(출처: 시장 조사 보고서).
2. 실현 가능성  
   1) 제품 및 서비스의 개발 방안  

   - 본 제품은 나무병원 및 조경업체가 수목의료를 위한 약품과 자재를 보다 효율적으로 관리할 수 있도록 돕기 위해 개발되었습니다. 현재 나무의사용 EMR 시스템은 헬스케어 전문기업 시야인사이트와 한양대학교 공과대학원과 협력하여 공동으로 개발하였으며, 이는 진단 및 처방 정보의 체계적인 기록과 관리를

In [49]:
# 저장할 폴더 경로
folder_path = "고객 사업정보"
os.makedirs(folder_path, exist_ok=True)  # 폴더가 존재하지 않으면 생성

# 파일 이름 생성
base_filename = f"{아이템_명} 사업계획서.jsonl"
filename = os.path.join(folder_path, base_filename)  # 폴더 경로와 파일 이름 결합
counter = 1

# 같은 이름의 파일이 존재할 경우 숫자를 붙여서 파일 이름 변경
while os.path.exists(filename):
    filename = os.path.join(folder_path, f"{아이템_명} 사업계획서 ({counter}).jsonl")
    counter += 1

# JSON Lines 형식으로 파일에 저장
with open(filename, 'a', encoding='utf-8') as f:
    json.dump({"사업계획서": 출력_사업계획서_초안}, f, ensure_ascii=False)
    f.write('\n')

print(f"입력된 정보가 {filename} 파일에 저장되었습니다.")



TypeError: Object of type function is not JSON serializable